In [171]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/shell-ultra-9/train_ultra_featured.csv
/kaggle/input/shell-ultra-9/test_ultra_featured.csv
/kaggle/input/shelladvanced/test_featured.csv
/kaggle/input/shelladvanced/train_featured.csv


In [172]:
!pip install tabpfn

In [173]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, KFold
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_absolute_percentage_error, make_scorer
from tabpfn import TabPFNRegressor

In [174]:
import optuna
import pandas as pd
import numpy as np
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from tabpfn import TabPFNRegressor

In [175]:
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

In [176]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import cross_val_score, StratifiedKFold
from tabpfn import TabPFNRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_absolute_percentage_error, make_scorer

In [177]:
import warnings

warnings.filterwarnings('ignore')

In [178]:
from sklearn.multioutput import RegressorChain
from tabpfn import TabPFNRegressor
import numpy as np

In [179]:
# Evaluation functions
def calculate_mape(y_true, y_pred):
    """Use sklearn's MAPE implementation."""
    return mean_absolute_percentage_error(y_true, y_pred)

def calculate_leaderboard_score(mape_cost, leaderboard_type='public'):
    """Leaderboard score: max[10, 100 - (90 × MAPE / Ref Cost)]"""
    ref_cost = 2.72  # Public leaderboard reference cost
    score = max(10, 100 - (90 * mape_cost) / ref_cost)
    return score

def evaluate_model(y_true, y_pred, leaderboard_type='public'):
    """Evaluate model using MAPE and leaderboard score."""
    mape = calculate_mape(y_true, y_pred)
    score = calculate_leaderboard_score(mape, leaderboard_type)
    return score, mape

# Create scorer for cross-validation (negative because sklearn maximizes scores)
def leaderboard_scorer(y_true, y_pred):
    score, _ = evaluate_model(y_true, y_pred)
    return score

custom_scorer = make_scorer(leaderboard_scorer, greater_is_better=True)


In [180]:
# 1. CRUCIAL: Define the order of the chain.
# This order is critical and must be determined through experimentation or domain knowledge.
# This is just a random example showing the format. The numbers correspond to the column indices of your y_train.
# Let's say y_train has 10 columns (0 to 9).
custom_order = [4,2,9,8,7,5,1,0,3,6] 
# A good starting point might be an order based on feature correlation with each target.


In [181]:
train = pd.read_csv('/kaggle/input/shelladvanced/train_featured.csv')
test = pd.read_csv('/kaggle/input/shelladvanced/test_featured.csv')

In [182]:
train

,Component1_fraction,Component2_fraction,Component3_fraction,Component4_fraction,Component5_fraction,Component1_Property1,Component2_Property1,Component3_Property1,Component4_Property1,Component5_Property1,...,Component5_Prop_1_x_Fraction,Component5_Prop_2_x_Fraction,Component5_Prop_3_x_Fraction,Component5_Prop_4_x_Fraction,Component5_Prop_5_x_Fraction,Component5_Prop_6_x_Fraction,Component5_Prop_7_x_Fraction,Component5_Prop_8_x_Fraction,Component5_Prop_9_x_Fraction,Component5_Prop_10_x_Fraction
0,0.21,0.00,0.42,0.25,0.12,-0.021782,1.981251,0.020036,0.140315,1.032029,...,0.123843,0.025934,-0.369953,-0.169206,0.035606,0.208956,0.026548,0.061599,-0.076367,-0.327471
1,0.02,0.33,0.19,0.46,0.00,-0.224339,1.148036,-1.107840,0.149533,-0.354000,...,-0.000000,0.000000,-0.000000,-0.000000,-0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.08,0.08,0.18,0.50,0.16,0.457763,0.242591,-0.922492,0.908213,0.972003,...,0.155521,-0.108521,0.230064,-0.007537,-0.488454,0.293861,-0.106395,0.166308,0.019813,-0.067890
3,0.25,0.42,0.00,0.07,0.26,-0.577734,-0.930826,0.815284,0.447514,0.455717,...,0.118486,0.053903,-0.347874,-0.252382,0.277352,-0.067915,-0.025092,-0.331107,-0.253182,-0.149872
4,0.26,0.16,0.08,0.50,0.00,0.120415,0.666268,-0.626934,2.725357,0.392259,...,0.000000,0.000000,0.000000,0.000000,0.000000,-0.000000,0.000000,0.000000,-0.000000,-0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0.50,0.12,0.00,0.26,0.12,0.279523,-0.054170,-0.391227,0.400222,1.032029,...,0.123843,0.152777,0.080635,-0.303384,0.149685,-0.007602,0.132469,0.001480,0.091998,0.076943
1996,0.19,0.31,0.00,0.37,0.13,-0.887185,0.610050,0.178606,1.083154,-2.822749,...,-0.366957,0.124486,-0.026678,-0.074217,-0.029339,0.110352,-0.129036,-0.067662,-0.329877,0.051643
1997,0.38,0.06,0.14,0.31,0.11,0.568978,-0.196759,-0.646318,-0.980070,1.032029,...,0.113523,-0.116252,0.061164,0.027390,0.072859,0.084863,-0.065236,0.176359,0.172434,-0.031314
1998,0.50,0.16,0.00,0.18,0.16,-0.067453,0.321977,-0.137535,0.238507,0.017455,...,0.002793,0.081606,-0.027888,-0.170876,0.115188,-0.347358,0.108605,-0.115872,-0.025832,0.030961


In [183]:
test

,ID,Component1_fraction,Component2_fraction,Component3_fraction,Component4_fraction,Component5_fraction,Component1_Property1,Component2_Property1,Component3_Property1,Component4_Property1,...,Component5_Prop_1_x_Fraction,Component5_Prop_2_x_Fraction,Component5_Prop_3_x_Fraction,Component5_Prop_4_x_Fraction,Component5_Prop_5_x_Fraction,Component5_Prop_6_x_Fraction,Component5_Prop_7_x_Fraction,Component5_Prop_8_x_Fraction,Component5_Prop_9_x_Fraction,Component5_Prop_10_x_Fraction
0,1,0.18,0.05,0.32,0.37,0.08,-0.177804,-0.741219,0.769821,-0.877069,...,0.048225,-0.175216,-0.096329,0.021203,-0.047328,-0.006673,-0.130548,-0.053605,0.103834,-0.062165
1,2,0.00,0.50,0.00,0.37,0.13,2.501354,0.177344,-0.498739,-0.196742,...,-0.252650,0.041821,0.022839,0.208719,0.019826,-0.009459,0.160683,0.076648,0.070652,-0.172775
2,3,0.16,0.00,0.17,0.50,0.17,1.547324,0.891479,0.030627,-0.368678,...,-0.050104,-0.150151,0.244443,-0.325301,0.261247,0.106982,0.024557,-0.035441,0.015899,-0.030133
3,4,0.50,0.00,0.17,0.16,0.17,-0.424427,1.016862,-1.182979,-0.854225,...,-0.141132,-0.065700,-0.317172,-0.067977,-0.054649,-0.336525,-0.077240,0.299038,-0.129262,0.293438
4,5,0.00,0.00,0.50,0.50,0.00,-0.187062,-0.762173,-0.473660,2.074087,...,0.000000,0.000000,-0.000000,0.000000,0.000000,-0.000000,0.000000,0.000000,-0.000000,-0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,496,0.44,0.01,0.08,0.41,0.06,1.036797,1.415667,0.793302,-0.446630,...,0.023731,-0.015241,-0.026058,0.078950,0.011270,-0.123370,-0.051994,-0.025862,-0.080681,0.050301
496,497,0.19,0.47,0.03,0.23,0.08,-1.305137,-1.520941,-0.989537,0.903203,...,0.082562,-0.015927,0.115032,0.017782,-0.053473,-0.099212,-0.099912,0.082861,0.029386,-0.132658
497,498,0.43,0.01,0.12,0.21,0.23,0.806590,0.607324,0.359058,0.283394,...,0.237367,0.292823,-0.179625,0.359684,0.046969,-0.320851,-0.267933,0.054265,-0.371135,-0.005065
498,499,0.03,0.04,0.42,0.42,0.09,-0.792140,0.674275,-1.783487,0.848296,...,0.014832,0.114583,-0.110158,-0.107278,-0.115154,0.009203,-0.107608,0.158314,-0.172139,-0.069779


In [184]:
feature_columns = train.columns
target_columns = [col for col in train.columns if 'BlendProperty' in col]
feature_columns=train.columns.drop(target_columns)

In [185]:
# Separate features and target
# Assuming last column is target, adjust if needed
X_train = train[feature_columns]  # All columns except last
y_train = train[target_columns]   # Last column as target

X_test = test[feature_columns]


In [188]:
import re

In [189]:
regex_pattern = r'^Component\d+_(fraction|Property\d+)$'

# 2. Filter the columns using the pattern
selected_columns = [col for col in feature_columns  if re.match(regex_pattern, col)]

In [190]:
# we need to remove outliers from training set.
import pandas as pd
import numpy as np
from sklearn.ensemble import IsolationForest

# Assume X_train and y_train are your loaded training DataFrames

# 1. Initialize IsolationForest
# The 'contamination' parameter is the most important one to set.
# It represents the percentage of outliers you expect in your data.
# Start with a small value like 0.01 or 0.02. 'auto' is also an option.
# random_state is for reproducibility.
iso_forest = IsolationForest(
    n_estimators=200,
    contamination=0.1, # Expect 1% of the data to be outliers
    random_state=100
)

# 2. Fit the model and predict the outliers on the training data
# It returns 1 for inliers and -1 for outliers.
outlier_preds = iso_forest.fit_predict(X_train)

# 3. Identify and count the outliers"
outlier_count = (outlier_preds == -1).sum()
print(f"Original number of training samples: {len(X_train)}")
print(f"Number of outliers identified: {outlier_count}")

# 4. Remove the outliers from both X_train and y_train
# It's crucial to filter both to keep them aligned.
X_train_no_outliers = X_train[outlier_preds == 1]
y_train_no_outliers = y_train[outlier_preds == 1]

print(f"New number of training samples after removing outliers: {len(X_train_no_outliers)}")

Original number of training samples: 2000
Number of outliers identified: 200
New number of training samples after removing outliers: 1800


In [191]:
train.shape,test.shape

((2000, 115), (500, 106))

In [192]:
X_train=X_train_no_outliers
y_train=y_train_no_outliers

In [193]:
X_train

,Component1_fraction,Component2_fraction,Component3_fraction,Component4_fraction,Component5_fraction,Component1_Property1,Component2_Property1,Component3_Property1,Component4_Property1,Component5_Property1,...,Component5_Prop_1_x_Fraction,Component5_Prop_2_x_Fraction,Component5_Prop_3_x_Fraction,Component5_Prop_4_x_Fraction,Component5_Prop_5_x_Fraction,Component5_Prop_6_x_Fraction,Component5_Prop_7_x_Fraction,Component5_Prop_8_x_Fraction,Component5_Prop_9_x_Fraction,Component5_Prop_10_x_Fraction
0,1.792148,1.756535,1.826537,1.798790,1.777041,1.752768,2.050731,1.759988,1.780471,1.920499,...,1.777690,1.761002,1.690535,1.726888,1.762663,1.791974,1.761108,1.767114,1.743263,1.698339
1,1.759982,1.811943,1.788811,1.832955,1.756535,1.717035,1.937355,1.544258,1.782023,1.693473,...,1.756535,1.756535,1.756535,1.756535,1.756535,1.756535,1.756535,1.756535,1.756535,1.756535
2,1.770252,1.770252,1.787138,1.839332,1.783783,1.832597,1.797563,1.583061,1.902189,1.911664,...,1.783030,1.737622,1.795485,1.755233,1.668438,1.806023,1.737996,1.784842,1.759950,1.744745
3,1.798790,1.826537,1.756535,1.768548,1.800444,1.651462,1.581349,1.888223,1.830956,1.832269,...,1.776784,1.765798,1.694598,1.711986,1.803307,1.744741,1.752194,1.697673,1.711841,1.730320
4,1.800444,1.783783,1.770252,1.839332,1.756535,1.777111,1.865413,1.641982,2.142145,1.822061,...,1.756535,1.756535,1.756535,1.756535,1.756535,1.756535,1.756535,1.756535,1.756535,1.756535
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1.839332,1.777041,1.756535,1.800444,1.777041,1.803664,1.747139,1.686604,1.823348,1.920499,...,1.777690,1.782569,1.770360,1.702737,1.782049,1.755222,1.779147,1.756791,1.772293,1.769731
1996,1.788811,1.808671,1.756535,1.818455,1.778730,1.590285,1.856670,1.786904,1.927962,1.088422,...,1.691087,1.777799,1.751919,1.743639,1.751457,1.775407,1.734006,1.744785,1.697899,1.765411
1997,1.820077,1.766840,1.780418,1.808671,1.775348,1.850234,1.721976,1.638223,1.571167,1.920499,...,1.775944,1.736261,1.767039,1.761253,1.769035,1.771080,1.745209,1.786528,1.785870,1.751114
1998,1.839332,1.783783,1.756535,1.787138,1.783783,1.744822,1.810632,1.732504,1.796886,1.759544,...,1.757017,1.770525,1.751709,1.726591,1.776226,1.694693,1.775111,1.736328,1.752066,1.761866


In [194]:
X_test

,Component1_fraction,Component2_fraction,Component3_fraction,Component4_fraction,Component5_fraction,Component1_Property1,Component2_Property1,Component3_Property1,Component4_Property1,Component5_Property1,...,Component5_Prop_1_x_Fraction,Component5_Prop_2_x_Fraction,Component5_Prop_3_x_Fraction,Component5_Prop_4_x_Fraction,Component5_Prop_5_x_Fraction,Component5_Prop_6_x_Fraction,Component5_Prop_7_x_Fraction,Component5_Prop_8_x_Fraction,Component5_Prop_9_x_Fraction,Component5_Prop_10_x_Fraction
0,1.787138,1.765130,1.810309,1.818455,1.770252,1.725358,1.619609,1.881319,1.592345,1.855539,...,1.764826,1.725819,1.739765,1.760189,1.748331,1.755382,1.733739,1.747238,1.774302,1.745745
1,1.756535,1.839332,1.756535,1.818455,1.778730,2.115495,1.786693,1.666497,1.721979,1.347780,...,1.711937,1.763729,1.760470,1.791935,1.759952,1.754901,1.783898,1.769681,1.768659,1.726253
2,1.783783,1.756535,1.785462,1.839332,1.785462,1.993292,1.899689,1.761809,1.690770,1.704313,...,1.747848,1.730271,1.797870,1.698736,1.800650,1.774836,1.760766,1.750398,1.759276,1.751319
3,1.839332,1.756535,1.785462,1.783783,1.785462,1.680438,1.918274,1.528088,1.596982,1.601839,...,1.731868,1.745128,1.700222,1.744730,1.747056,1.696681,1.743111,1.806873,1.733966,1.805953
4,1.756535,1.756535,1.839332,1.839332,1.756535,1.723707,1.615452,1.671224,2.062603,1.879340,...,1.756535,1.756535,1.756535,1.756535,1.756535,1.756535,1.756535,1.756535,1.756535,1.756535
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,1.829751,1.758260,1.770252,1.824926,1.766840,1.921197,1.975191,1.884891,1.676293,1.822589,...,1.760624,1.753900,1.752026,1.770073,1.758479,1.735006,1.747518,1.752060,1.742508,1.765182
496,1.788811,1.834553,1.761701,1.795475,1.770252,1.501228,1.451940,1.569198,1.901441,1.920499,...,1.770688,1.753782,1.776200,1.759600,1.747261,1.739259,1.739136,1.770739,1.761596,1.733366
497,1.828145,1.758260,1.777041,1.792148,1.795475,1.886907,1.856244,1.816678,1.804302,1.920499,...,1.796697,1.805852,1.725033,1.816780,1.764611,1.699550,1.709175,1.765860,1.690317,1.755660
498,1.761701,1.763417,1.826537,1.826537,1.771953,1.609476,1.866652,1.388504,1.893207,1.784589,...,1.759092,1.776124,1.737334,1.737841,1.736454,1.758123,1.737783,1.783500,1.726366,1.744415


In [195]:
min_val = y_train.min().min() 
# Add a small epsilon (1) to ensure all values are > 0 after shifting
shift = abs(min_val) + 1

print(f"Minimum value in y_train is: {min_val:.4f}")
print(f"Using a shift value of: {shift}")

# 1. Apply the shift and then the log transform
y_train_transformed = np.log1p(y_train + shift)


Minimum value in y_train is: -3.6211
Using a shift value of: 4.621079932379008


In [196]:
model = MultiOutputRegressor(TabPFNRegressor(device='cuda'))
print("\nFitting model on shifted-and-logged targets...")
model.fit(X_train, y_train_transformed)
# model.fit(X_train, y_train)


Fitting model on shifted-and-logged targets...


MultiOutputRegressor(estimator=TabPFNRegressor(device='cuda'))

In [197]:
print("Making predictions...")
transformed_predictions = model.predict(X_test)

Making predictions...


In [198]:
final_predictions = np.expm1(transformed_predictions) - shift
# final_predictions = model.predict(X_test)
final_predictions_df = pd.DataFrame(final_predictions, columns=target_columns)

In [199]:
final_predictions_df['ID']=test['ID']

In [200]:
final_predictions_df

,BlendProperty1,BlendProperty2,BlendProperty3,BlendProperty4,BlendProperty5,BlendProperty6,BlendProperty7,BlendProperty8,BlendProperty9,BlendProperty10,ID
0,0.163154,0.311771,0.718825,0.624286,0.351751,0.733582,0.688111,0.341935,-0.336933,0.328554,1
1,-0.810663,-0.563935,-1.175026,0.022408,-0.728502,-0.118439,-1.168720,-1.053155,-0.969693,-0.000827,2
2,1.777750,1.139267,1.053353,1.058774,2.490036,1.837607,1.008721,2.020516,0.729743,2.252620,3
3,-0.449485,0.312607,0.803555,-0.718658,1.902359,-0.465666,0.750749,1.641893,0.870730,-0.971623,4
4,0.173930,-1.174822,1.085490,0.486665,2.481368,0.265960,1.059388,-0.009676,-0.432758,1.056813,5
...,...,...,...,...,...,...,...,...,...,...,...
495,0.207924,-0.909070,1.208858,-0.242666,-0.273782,-0.719590,1.151661,-0.547663,-1.411229,-0.473084,496
496,-2.175349,-1.330848,-1.068008,-2.296855,-0.626490,-2.394543,-1.053443,-1.905941,-1.286872,-1.348949,497
497,2.016032,2.267026,0.312740,1.141640,-0.034360,0.658002,0.276921,1.037652,0.295006,0.486671,498
498,-0.124332,0.809366,1.636301,-1.391518,-0.911981,0.183698,1.970482,0.590534,0.199015,1.278523,499


In [201]:
final_predictions_df.to_csv('outliers_15.csv',index=False)